### Classification based on extracted features

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

#from keras.applications.vgg16 import VGG16 as BaseModel
#from keras.applications.vgg16 import preprocess_input

from keras.applications.xception import Xception as BaseModel
from keras.applications.xception import preprocess_input

from keras.preprocessing import image
from keras.models import Model
import numpy as np

Using TensorFlow backend.
/home/daryna/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/daryna/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/daryna/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/daryna/.local/lib/python3.7/site-packages/tensorflow/python

In [2]:
model = BaseModel(weights='imagenet', include_top=True)


91889664/91884032 [==============================] - 30s 0us/step


In [3]:
model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [4]:
features_model = Model(input=model.input, output=model.layers[-2].output)

/home/daryna/anaconda3/envs/ml_ukma/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("av...)`
  """Entry point for launching an IPython kernel.


In [5]:
features_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [6]:
test_dir = 'data/training_folder/test'
val_dir = 'data/training_folder/val'
train_dir = 'data/training_folder/train'
image_size = tuple(model.input.shape[1:3].as_list()) ###(224,224)

In [7]:
from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=image_size,
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=image_size,
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=image_size,
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

test_filenames = test_generator.filenames
val_filenames = val_generator.filenames
train_filenames = train_generator.filenames

test_nb_samples = len(test_filenames)
val_nb_samples = len(val_filenames)
train_nb_samples = len(train_filenames)

predict_test = features_model.predict_generator(test_generator,steps=test_nb_samples)
predict_val = features_model.predict_generator(val_generator,steps=val_nb_samples)
predict_train = features_model.predict_generator(train_generator,steps=train_nb_samples)

Found 340 images belonging to 17 classes.
Found 340 images belonging to 17 classes.
Found 680 images belonging to 17 classes.



In [8]:
predict_train.shape

(680, 2048)

Now we'll use classical ML algorithm on extracted features

Let us use only the first two classes

In [9]:
features_test = predict_test[test_generator.classes <=1]
features_train = predict_val[val_generator.classes <=1]

y_test = test_generator.classes[test_generator.classes <=1]
y_train = val_generator.classes[val_generator.classes <=1]

In [10]:
from sklearn.linear_model import LogisticRegression

In [11]:
clf = LogisticRegression()

In [12]:
clf.fit(features_train, y_train)

LogisticRegression()

In [13]:
pred = clf.predict(features_test)

In [14]:
from sklearn.metrics import accuracy_score
accuracy_score(pred, y_test)

0.925

For multiple classes

In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [16]:
n_neighbors = 20

In [17]:
#clf = KNeighborsClassifier(n_neighbors=n_neighbors)
#clf = RandomForestClassifier(random_state=42)
clf = MLPClassifier()

In [18]:
features_test = predict_test
features_train = predict_train

y_test = test_generator.classes
y_train = train_generator.classes

In [19]:
clf.fit(features_train, y_train)

MLPClassifier()

In [20]:
pred = clf.predict(features_test)

In [21]:
accuracy_score(pred, y_test)

0.9029411764705882